# AB Testi - Ortalama Oturum Süresi

Hoş geldiniz! Bu çalışmada, mevcut bir ürünü iyileştirmek için bir eylem seçmek üzere bir AB testi gerektiren iki vaka sunulacak. Sürekli ve oransal bir metrik için AB testi gerçekleştirecez. Bunun için, örneklerden ilgili bilgileri tahmin eden, her vaka için ilgili istatistiği hesaplayan ve sıfır hipotezini reddedip reddetmemeye karar veren işlevleri tanımlayacaz.

Başlayalım!

# Ana Hatlar
- [ 1 - Giriş](#1)
- [ 2 - Verileri keşfetme ve işleme](#2)
- [ 3 - Teoriyi yeniden gözden geçirme](#3)
- [ 4 - Adım adım hesaplama](#4)
- [ Alıştırma 1](#ex01)
- [ Alıştırma 2](#ex02)
- [ Alıştırma 3](#ex03)
- [ Alıştırma 4](#ex04)
- [ Alıştırma 5](#ex05)

In [3]:
import math
import numpy as np
import pandas as pd
from scipy import stats

In [4]:
import w4_unittest

<a name="1"></a>
## 1 - Giriş

Blog benzeri bir formatta makine öğrenimi içeriği sağlayan bir web siteniz olduğunu varsayalım. Yakın zamanda, benzer web sitelerinin arka plan için belirli bir renk paleti kullanarak etkileşimlerini artırabileceğini iddia eden bir makale gördünüz. Bu değişikliğin uygulanması oldukça kolay göründüğünden, bu değişikliğin kullanıcılarınızı web sitenizde daha fazla zaman geçirmeye yönlendirip yönlendirmediğini görmek için bir AB testi çalıştırmaya karar verdiniz.

Değerlendirmeye karar verdiğiniz ölçüm, kullanıcılarınızın web sitenizde ortalama olarak ne kadar zaman harcadığını ölçen `ortalama oturum süresi`dir. Bu ölçümün şu anda değeri 30,87 dakikadır.

Başka bir değerlendirme yapmadan, kullanıcılarınızı rastgele iki segmente ayırarak testi 20 gün boyunca çalıştırmaya karar verdiniz:
- `kontrol`: Bu kullanıcılar orijinal web sitenizi görmeye devam edecek.

- `değişim`: Bu kullanıcılar web sitenizi yeni arka plan renkleriyle görecek.

<a name="2"></a>
## 2 - Verileri keşfetme ve işleme



In [2]:
# pd.read_csv kullanarak testten verileri yükleyin
data = pd.read_csv("background_color_experiment.csv")

# İlk 10 satırı yazdır
data.head(10)

,user_id,user_type,session_duration
0,BM3C0BJ7CS,variation,15.528769
1,MJWN6XNH6L,variation,32.287590
2,46ZPHHABLS,variation,43.718217
3,OHA298DHUG,variation,49.519702
4,AKJ77X6F4A,control,61.709028
5,BFNWMGU6DX,variation,71.779283
6,UFO2V8ZKFB,variation,23.291835
7,4CEIM3VRS9,control,25.219461
8,90AGF68FF8,control,26.240482
9,R3DQFO6068,variation,20.780244


In [5]:
print(f"Veri kümesinin boyutu: {len(data)}")

Veri kümesinin boyutu: 4186


Veriler her kullanıcı için ortalama oturum süresini ve etkileşimde bulundukları web sitesinin sürümünü gösterir. Daha kolay hesaplamalar için her iki segmenti ayırmak için aşağıdaki hücreyi çalıştırarak Pandas veri çerçevesini dilimleyebilirsiniz.

In [7]:
# Verileri iki gruptan ayırın (sd oturum süresini ifade eder)
control_sd_data = data[data["user_type"]=="control"]["session_duration"]
variation_sd_data = data[data["user_type"]=="variation"]["session_duration"]

print(f"{len(control_sd_data)} kullanıcı orijinal web sitesini ortalama {control_sd_data.mean():.2f} dakika geçirdi\n")
print(f"{len(variation_sd_data)} kullanıcı yeni web sitesini ortalama {variation_sd_data.mean():.2f} dakika geçirdi")

2069 kullanıcı orijinal web sitesini ortalama 32.92 dakika geçirdi

2117 kullanıcı yeni web sitesini ortalama 33.83 dakika geçirdi


Bölünmenin mükemmel bir şekilde dengelenmediğine dikkat edin. Bu, AB testinde yaygındır çünkü kullanıcıların her gruba atanma şekliyle ilişkili bir rastgelelik vardır.

İlk bakışta arka plandaki değişikliğin kullanıcıları web sitenizde daha uzun süre kalmaya yönlendirdiği gibi görünüyor. Ancak bu verilerden yüzeysel sonuçlar çıkarmaktan daha iyisini biliyorsunuz, bu yüzden bu iki segmentin **ortalamaları** arasında önemli bir fark olup olmadığını bilmek için bir hipotez testi yapmaya karar veriyorsunuz.

<a name="3"></a>
## 3 - Teoriyi yeniden ele almak

Teoriyi yeniden ele alalım ve bunu bu probleme uygulayalım. Teoriye güveniyorsanız ve bir revizyona ihtiyacınız olmadığını düşünüyorsanız, bu bölümü atlayıp doğrudan 1.4'e geçebilirsiniz!

Unutmayın, işiniz web sitesinin arka plan rengini değiştirmenin ziyaretçilerin web sitesinde geçirdiği sürede bir artışa yol açıp açmadığını ölçmektir. Bunu hipotez testi olarak yeniden yazarsak, **sıfır hipotezi**, değişikliğin bir ziyaretçinin geçirdiği zamanı etkilemediğidir. Değişkenlere isim verelim:

- $\mu_c$ **kontrol grubundaki** bir kullanıcının web sitesinde geçirdiği ortalama zamandır. **Kontrol grubunun** arka plan renginde değişiklik olmadan web sitesine erişen grup olduğunu hatırlayın.
- $\mu_v$ **varyasyon gruplarındaki** bir kullanıcının web sitesinde geçirdiği ortalama zamandır. **varyasyon grubunun** web sitesine **güncellenmiş arka plan rengiyle** erişen gruplar olduğunu hatırlayın.

Ayrıca, amacınızın arka plan renginin bir ziyaretçinin web sitesinde geçirdiği sürede **artış** yaratıp yaratmadığını ölçmek olduğunu hatırlayın. Bu deneyi bir hipotez testi olarak yazarsak, **sıfır hipotezi** $H_0: \mu_c = \mu_v$ ve **alternatif hipotez** $H_1: \mu_v > \mu_c$ veya eşdeğer olarak $H_1: \mu_v - \mu_c > 0$ olur.

Bu nedenle, test edeceğiniz hipotez şudur:

$$H_0: \mu_v = \mu_c \quad \text{vs.} \quad H_1: \mu_v - \mu_c > 0$$

Ortalama sürede bir artış aradığınız için bu **sağ kuyruklu** bir testtir. Yukarıda gördüğünüz gibi, grup başına 2000'den fazla kullanıcınız var, bu çok miktarda veri olduğundan, her grup için **ortalama sürenin** normal bir dağılıma uyduğu Merkezi Limit Teoremi'ne güvenmek mantıklıdır. Bu sonucun, grubun **ortalama süresi** için olduğunu ve her kullanıcının harcadığı sürenin normal bir dağılıma uymadığını unutmayın. Bir kullanıcının web sitenizde geçirdiği sürenin tam dağılımını bilmiyorsunuz, ancak CLT, yeterli veri toplarsak ortalama sürelerinin, ortalaması bir kullanıcının web sitesinde geçirdiği ortalama süre olan normal bir dağılıma çok yakın olacağını garanti eder. O zaman iki yeni nicelik tanımlayalım:

- $\overline{X}_c$ - kontrol grubu **örnek ortalaması**.
- $\overline{X}_v$ - varyasyon grubu **örnek ortalaması**.
- $n_c$ - kontrol grubu **boyutu**.
- $n_v$ - varyasyon grubu **boyutu**.

Yani, Merkezi Limit Teoremi'ne göre, şunu varsayabilirsiniz

- $$\overline{X}_c \sim N\left(\mu_c, \left(\frac{\sigma_c}{\sqrt{n_c}}\right)^2\right)$$
- $$\overline{X}_v \sim N\left(\mu_v, \left(\frac{\sigma_v}{\sqrt{n_v}}\right)^2\right)$$

Normallik varsayımlarımızla, $\overline{X}_v - \overline{X}_c$'nin de normal bir dağılım izlediğini unutmayın. Yani, eğer $H_0$ doğruysa, o zaman $\mu_c = \mu_v$ ve $\mu_v - \mu_c = 0$, dolayısıyla:

$$\overline{X}_c - \overline{X}_v \sim N\left(\mu_v - \mu_c, \left(\dfrac{\sigma_v}{\sqrt{n_v}}\right)^2 + \left(\dfrac{\sigma_c}{\sqrt{n_c}}\right)^2\right) = N\left(0, \left(\dfrac{\sigma_v}{\sqrt{n_v}}\right)^2 + \left(\dfrac{\sigma_c}{\sqrt{n_c}}\right)^2\right)$$

Ya da eşdeğer olarak:

$$\frac{\left( \overline{X}_v - \overline{X}_c \right)}{\sqrt{\left(\frac{\sigma_v}{\sqrt{n_v}}\right)^2 + \left(\frac{\sigma_c}{\sqrt{n_c}}\right)^2}} \sim N(0, 1)$$

Ancak, $\sigma_v$ ve $\sigma_c$ için kesin değerleri **bilmediğinizi** unutmayın, çünkü bunlar **popülasyon standart sapması** ve bir örneklemle çalışıyorsunuz, bu nedenle yapabileceğiniz en iyi şey **örneklem standart sapmasını** hesaplamaktır. Bu nedenle $\sigma_c$ ve $\sigma_v$ değerlerini sırasıyla örneklem standart sapması, $s_c$ ve $s_v$ ile değiştirmelisiniz. Ayrıca  popülasyon standart sapmasının örnek standart sapmasıyla değiştirilmesinin rastgele değişkeni Normal'den t-öğrenciye dönüştürdüğünü biliyoruz:

$$t = \frac{\left( \overline{X}_v - \overline{X}_c \right)}{\sqrt{\left(\frac{s_v}{\sqrt{n_v}}\right)^2 + \left(\frac{s_c}{\sqrt{n_c}}\right)^2}} \sim t_d$$

Burada $d$ bu senaryo için **serbestlik derecesidir**. Her iki grubun da aynı standart sapmaya sahip olduğunu varsayarsak, o zaman $d = n_c + n_v - 2$ olur, ancak bu varsayımı destekleyen bir argüman yoktur, bu nedenle serbestlik derecesi formülü biraz daha karmaşık hale gelir:

$$d = \frac{\left[\frac{s_{v}^2}{n_v} + \frac{s_{c}^2}{n_c} \right]^2}{\frac{(s_{v}^2/n_v)^2}{n_v-1} + \frac{(s_{c}^2/n_c)^2}{n_c-1}}$$

$t_d$ için gerçek değeri elde ettiğinizde, belirli bir önem düzeyi $\alpha$ ile, bu değerin aşağıdaki değerler aralığına girip girmediğine karar verebilirsiniz:

$t$-öğrenci dağılımında meydana gelme olasılığı (burada 'olasılık' önem seviyenizle ilişkilidir). Bu adımı gerçekleştirmek için,

$$p = P(t_d > t | H_0)$$
 olacak şekilde $p$ değerini bulmalısınız

Bu değer önem seviyenizden $\alpha$ düşükse, o zaman **sıfır hipotezini reddedersiniz**, çünkü bu, $H_0$ doğruysa meydana gelme olasılığı çok düşük bir değer gözlemlediğiniz anlamına gelir (burada olası olmayan, belirlediğiniz önem seviyesinden düşük anlamına gelir).

Ayrıca, $P(t_d \leq t)$'nin $x = t$ noktasında $d$ serbestlik derecesine sahip $t$-öğrenci dağılımı için $\text{CDF}$ (kümülatif dağılım fonksiyonu) olduğunu unutmayın, bu nedenle $P(t_d > t)$'yi hesaplamak için şunu hesaplayabilirsiniz:

$$P(t_d > t) = 1 - \text{CDF}_{t_d}(t)$$

$P(t_d \leq t) + P(t_d > t) = 1$ olduğundan

<a name="4"></a>
## 4 - Adım adım hesaplama

Yukarıda tartışılan her şeyi özetlersek:

Hipotez testi şu şekilde verilir:

$$H_0: \mu_v = \mu_c \quad \text{vs.} \quad H_1: \mu_v - \mu_c > 0$$

Şunları hesaplamaya başlayacaksınız:

- $n_c$ ve $n_v$, sırasıyla kontrol ve varyasyon grubu boyutları.
- $\overline{X}_c$ ve $\overline{X}_v$, sırasıyla kontrol ve varyasyon grubundaki kullanıcılar tarafından harcanan ortalama zaman.
- $s_c$ ve $s_v$, sırasıyla kontrol ve varyasyon grubundaki kullanıcılar tarafından harcanan zamanın **örnek** standart sapması.

Bu miktarlar elinizdeyken, bir sonraki adımlar şunları hesaplamaktır:

- $d$, $t$-öğrenci dağılımının serbestlik derecesi, $t_d$.
- $t$ olarak adlandırılacak $t$-değeri.
- $t$-değeri için $t_d$ dağılımının $p$ değeri, yani $p = P(t_d > t | H_0)$ değeri.

Son olarak, belirli bir önem seviyesi $\alpha$ için, $p \leq \alpha$ olup olmadığına bağlı olarak $H_0$'ı reddedip reddetmeyeceğinize karar verebileceksiniz.

Hadi şimdi işe koyulalım! Kontrol ve varyasyon grupları için oturum sürelerini almak üzere aşağıdaki hücreyi çalıştırın.

In [8]:
# X_c kontrol grubu için oturum zamanını, X_v ise varyasyon grubu için oturum zamanını depolar. 
X_c = control_sd_data.to_numpy()
X_v = variation_sd_data.to_numpy()

In [9]:
print(f"X_c için ilk 10 giriş şunlardır:\n{X_c[:20]}\n")
print(f"X_v için ilk 10 giriş şunlardır:\n{X_v[:20]}\n")

X_c için ilk 10 giriş şunlardır:
[ 61.70902753  25.21946052  26.2404824   58.7480264  137.03680289
  19.92148102  18.8252202   75.25179496  38.27213776  29.17104128
  15.69643672  37.83860271  30.06843075  21.00318148  86.19711927
  46.96997965  46.47776713  14.83464105  17.70441365  26.44693676]

X_v için ilk 10 giriş şunlardır:
[15.52876878 32.28759003 43.7182168  49.51970242 71.77928343 23.29183517
 20.78024375 36.44129464 48.75034676 16.5952978  44.49566616 26.67006134
 34.43667579 20.72109411 19.60185277 41.74218978 19.74485294 32.62018094
 44.99513901 70.8916231 ]



<a name="ex01"></a>
### Alıştırma 1

Bu alıştırmada, `X_c` ve `X_d` için temel istatistikleri almak üzere bir fonksiyon yazacaz. Başka bir deyişle, bu fonksiyon, verilen bir numpy dizisi için şunları hesaplayacaktır:

- Boyutu (sizin durumunuzda, $n_c$ ve $n_v$).
- Ortalaması (sizin durumunuzda, $\overline{X}_c$ ve $\overline{X}_v$)
- Örneklem standart sapması (sizin durumunuzda, $s_c$ ve $s_v$)

Bu fonksiyon bir numpy dizisi girer ve `(n, x, s)` biçiminde bir tuple çıkarır; burada `n` numpy dizisinin boyutu, `x` ortalaması ve `s` **örneklem** standart sapmasıdır.

İpucu:
- Örnek standart sapmasının, varyans formülünde $N$ yerine $N-1$ koyarak hesaplandığını hatırlayın.
- `len` fonksiyonunu kullanarak bir dizi boyutunu hesaplayabilirsiniz.
- Numpy'deki herhangi bir dizinin ortalamasını hesaplamak için `.mean()` adlı bir yöntemi vardır.
- Numpy'deki herhangi bir dizinin standart sapmayı hesaplamak için `.std()` adlı bir yöntemi ve `ddof = 1` geçerseniz $N$ yerine $N-1$ kullanacağı `ddof` adlı bir parametresi vardır.

In [10]:
def get_stats(X):
    """
    Verilen bir veri kümesinin temel istatistiklerini hesaplar.

    Parametreler:
    X (numpy.array): Giriş verisi.

    Dönüş:
    tuple: Şu değerleri içeren bir demet döndürür:
        - n (int): Veri kümesindeki eleman sayısı.
        - x (float): Veri kümesinin ortalaması.
        - s (float): Veri kümesinin örneklem standart sapması.
    """
    
    # Veri kümesindeki eleman sayısını al
    n = len(X)
    # Veri kümesinin ortalamasını hesapla
    x = np.mean(X)
    # Veri kümesinin örneklem standart sapmasını hesapla
    s = np.std(X, ddof=1)
    
    return (n, x, s)


In [12]:
n_c, x_c, s_c = get_stats(X_c)
n_v, x_v, s_v = get_stats(X_v)

In [13]:
print(f"For X_c:\n\tn_c = {n_c}, x_c = {x_c:.2f}, s_c = {s_c:.2f} ")
print(f"For X_v:\n\tn_v = {n_v}, x_v = {x_v:.2f}, s_v = {s_v:.2f} ")

For X_c:
	n_c = 2069, x_c = 32.92, s_c = 17.54 
For X_v:
	n_v = 2117, x_v = 33.83, s_v = 18.24 


<a name="ex02"></a>
### Alıştırma 2

Bu alıştırmada $t$-öğrenci dağılımının serbestlik derecesi olan $d$'yi hesaplamak için bir fonksiyon uygulayacaksınız. Aşağıdaki formülle verilir:

$$d = \frac{\left[\frac{s_{c}^2}{n_c} + \frac{s_{v}^2}{n_v} \right]^2}{\frac{(s_{c}^2/n_c)^2}{n_c-1} + \frac{(s_{v}^2/n_v)^2}{n_v-1}}$$

Parantez içindekilere dikkat edin çünkü bunlar Python'un hesaplamayı hangi sırayla gerçekleştireceğini gösterecektir!

In [34]:
import numpy as np

def degrees_of_freedom(n_v, s_v, n_c, s_c):
    """
    İki örnek için serbestlik derecelerini hesaplar.

    Parametreler:
        n_v (int): Varyasyon örneğinin boyutu.
        s_v (float): Varyasyon örneğinin standart sapması.
        n_c (int): Kontrol örneğinin boyutu.
        s_c (float): Kontrol örneğinin standart sapması.

    Dönüş:
        numpy.float: Serbestlik derecesi.
    """
    
    # s_v^2/n_v hesapla
    s_v_n_v = np.square(s_v) / n_v
    
    # s_c^2/n_c hesapla
    s_c_n_c = np.square(s_c) / n_c
    
    # Payı hesapla
    numerator = np.square(s_c_n_c + s_v_n_v)
    
    # Paydada her iki terimi ayrı ayrı hesapla
    denominator = (np.square(s_c_n_c) / (n_c - 1)) + (np.square(s_v_n_v) / (n_v - 1))
    
    dof = numerator / denominator
        
    return dof


In [16]:
d = degrees_of_freedom(n_v, s_v, n_c, s_c)
print(f"Bu senaryoda t-öğrencinin serbestlik derecesi şudur: {d:.2f}")

Bu senaryoda t-öğrencinin serbestlik derecesi şudur: 4182.97


<a name="ex03"></a>
### Alıştırma 3

Bu alıştırmada, aşağıdaki şekilde verilen $t$ değerini hesaplayacaz:

$$t = \frac{\left( \overline{X}_v - \overline{X}_c \right)}{\sqrt{\left(\frac{s_v}{\sqrt{n_v}}\right)^2 + \left(\frac{s_c}{\sqrt{n_c}}\right)^2}} = \frac{\left( \overline{X}_v - \overline{X}_c \right)}{\sqrt{\frac{s_v^2}{n_v} + \frac{s_c^2}{n_c}}}$$

$\overline{X}_c$ ve $\overline{X}_v$ değerlerini sırasıyla $x_c$ ve $x_d$ değişkenlerinde sakladığınızı unutmayın.

In [ ]:
def t_value(n_v, x_v, s_v, n_c, x_c, s_c):

    # Daha önce yaptığınız gibi, kodu daha temiz hale getirmek için pay ve paydayı bölelim.
    # Ayrıca, s_c^2/n_c ve s_v^2/n_v'yi ayrı ayrı tekrar hesaplayalım.

    # s_v^2/n_v hesapla
    s_v_n_v = np.square(s_v) / n_v

    # s_c^2/n_c hesapla
    s_c_n_c = np.square(s_c) / n_c

    # t-istatistiği için payı hesapla
    numerator = x_v - x_c

    # t-istatistiği için paydayı hesapla
    denominator = np.sqrt(s_v_n_v + s_c_n_c)

    # t-değerini hesapla
    t = numerator / denominator

    return t

In [19]:
t = t_value(n_v, x_v, s_v, n_c, x_c, s_c)
print(f"Bu deneyin t değeri şudur: {t:.2f}")

Bu deneyin t değeri şudur: 1.64


##### __Expected Output__

```
The t-value for this experiment is: 1.64
```

<a name="ex04"></a>
### Alıştırma 4

Bu alıştırmada, belirli bir önem düzeyi $\alpha$ için $t_d$ için $p$ değerini hesaplayacaz. Bu deneyin sağ kuyruklu bir t-testi olduğunu unutmayın, çünkü arka plan renginin değişmesinin kullanıcıların web sitenizde geçirdiği zamanı artırıp artırmadığını araştırıyorsunuz.

Bu deneyde, $\alpha$ önem düzeyi için $p$ değeri şu şekilde verilir

$$p = P(t_d > t) = 1 - \text{CDF}_{t_d}(t)$$


Örnek:

Diyelim ki $d = 10$ serbestlik derecesine sahip $t$-öğrenci dağılımı için $1.21$ t değeri için CDF'yi hesaplamak istiyorsunuz.

In [20]:
t_10 = stats.t(df = 10)
cdf = t_10.cdf(1.21)
print(f"10 serbestlik derecesine ve t-değeri = 1.21'e sahip t-öğrenci dağılımının CDF'si veya eşdeğer olarak P(t_10 < 1.21) şuna eşittir: {cdf:.2f}")

10 serbestlik derecesine ve t-değeri = 1.21'e sahip t-öğrenci dağılımının CDF'si veya eşdeğer olarak P(t_10 < 1.21) şuna eşittir: 0.87


Bu, 10 serbestlik derecesine sahip bir $t$-öğrenci dağılımından örnekleme yaparken 1,21'den küçük bir değer gözlemleme olasılığınızın %87 olduğu anlamına gelir.

Tamam, şimdi $d$ serbestlik derecesine ve belirli bir $t$-değerine sahip $t$-öğrenci dağılımı için $p$-değerini hesaplayan bir fonksiyon yazmaya hazırız.

In [21]:
def p_value(d, t_value):
    """
    Verilen serbestlik derecesi ve t-değeri için p-değerini hesaplar.

    Parametreler:
        d (int): Serbestlik derecesi (degrees of freedom).
        t_value (float): Hesaplanmış t-değeri.

    Dönüş:
        float: Sağ kuyruklu t-testine göre p-değeri.
    """

    # Serbestlik derecesine sahip t-öğrenci dağılımını yükle
    t_d = stats.t(df=d)

    # p-değerini hesapla, P(t_d > t). Doğru ayarlamalarla t_d.cdf kullanmayı unutmayın.
    p = 1 - t_d.cdf(t_value)

    return p

In [23]:
print(f"t_15 için t-değeri = 1.10 olan p-değeri şudur: {p_value(15, 1.10):.4f}")
print(f"t_30 için t-değeri = 1.10 olan p-değeri şudur: {p_value(30, 1.10):.4f}")

t_15 için t-değeri = 1.10 olan p-değeri şudur: 0.1443
t_30 için t-değeri = 1.10 olan p-değeri şudur: 0.1400


<a name="ex05"></a>
### Alıştırma 5

Bu alıştırmada, $\alpha$ önem seviyesi verildiğinde $H_0$'ı kabul edip etmediğinize karar vermek için şimdiye kadar oluşturduğunuz tüm işlevleri toparlayacaz.

Hem kontrol hem de doğrulama gruplarını girecek ve buna göre `H_0$'ı reddet` veya `H_0'ı reddetme` çıktısını verecektir.

p değeri $\alpha$'dan **küçükse** $H_0$'ı **reddettiğinizi** unutmayın.

In [36]:

def make_decision(X_v, X_c, alpha = 0.05):
    """
    Verilen veri kümeleri (X_v, X_c) için t-testini yapar ve H_0 hipotezini reddetme veya reddetmeme kararını verir.
    
    Parametreler:
    X_v (numpy.array): Doğrulama grubunun veri kümesi.
    X_c (numpy.array): Kontrol grubunun veri kümesi.
    alpha (float): Önem seviyesi (default: 0.05).
    
    Dönüş:
    string: 'Reject H_0' veya 'Do not reject H_0' kararı.
    """
    
    # Doğrulama grubu ve kontrol grubu için temel istatistikleri hesapla
    n_v, x_v, s_v = get_stats(X_v)
    n_c, x_c, s_c = get_stats(X_c)
    
    # Serbestlik derecesini hesapla
    d = degrees_of_freedom(n_v, s_v, n_c, s_c)
    
    # t-değerini hesapla
    t = t_value(n_v, x_v, s_v, n_c, x_c, s_c)
    
    # p-değerini hesapla
    p = p_value(d, t)
    
    # Karar verme: p-değeri α'dan küçükse H₀'ı reddet
    if p < alpha:
        return 'H_0  reddet'
    else:
        return 'H_0 reddetmeyin'


In [38]:
alphas = [0.06, 0.05, 0.04, 0.01]
for alpha in alphas:
    print(f"{alpha} alfası için karar şudur: {make_decision(X_v, X_c, alpha = alpha)}")

0.06 alfası için karar şudur: H_0  reddet
0.05 alfası için karar şudur: H_0 reddetmeyin
0.04 alfası için karar şudur: H_0 reddetmeyin
0.01 alfası için karar şudur: H_0 reddetmeyin
